In [1]:
import numpy as np
import dimod
import matplotlib.pyplot as plt
import dwave_networkx as dnx
import networkx as nx
import minorminer
import minorminer.layout as mml
%matplotlib inline
import dwave.inspector
import matplotlib as mpl

In [2]:
import os

from dwave.system.samplers import DWaveSampler
from dwave.cloud.exceptions import *

try:
    qpu_advantage = DWaveSampler(solver={'topology__type': 'pegasus'},token=" ")
    qpu_2000q = DWaveSampler(solver={'topology__type': 'chimera'},token=" ")
    
    qpus = {'Advantage': qpu_advantage, 'DW-2000Q': qpu_2000q}

    print("Connected to Advantage {} and 2000Q {}.".format(qpu_advantage.solver.id, qpu_2000q.solver.id))
except SolverNotFoundError:
    print("Currently a pair of solvers are unavailable for sections comparing QPU technologies. Try those examples later.")

Connected to Advantage Advantage_system6.1 and 2000Q DW_2000Q_6.


In [3]:
from dwave.system import DWaveSampler

# Use a D-Wave system as the sampler
sampler = DWaveSampler(solver=dict(topology__type='chimera'),token="DEV-5fa1054cd25ed5a94ead77ab54c60e280f772f28")

print("QPU {} was selected.".format(sampler.solver.name))

QPU DW_2000Q_6 was selected.


In [4]:
#making a dictionary of sites and their corresponding 2chains. There are 4 2chains per unit cell
sites_2chains={}
sites_2chains.update({1:(1,5)})
sites_2chains.update({4:(10,15)})
sites_2chains.update({7:(130,135)})
sites_2chains.update({10:(137,141)})
print(sites_2chains)

{1: (1, 5), 4: (10, 15), 7: (130, 135), 10: (137, 141)}


In [5]:
#making a dictionary of sites and their corresponding 3chains. There are 8 3chains per unit cell
sites_3chains={}
sites_3chains.update({0:[(2,6),(6,14)]})
sites_3chains.update({2:[(7,3),(3,131)]})
sites_3chains.update({3:[(9,12),(12,20)]})
sites_3chains.update({5:[(13,8),(8,136)]})
sites_3chains.update({6:[(129,132),(132,140)]})
sites_3chains.update({8:[(133,128),(128,256)]})
sites_3chains.update({9:[(138,142),(142,150)]})
sites_3chains.update({11:[(143,139),(139,267)]})
print(sites_3chains)

{0: [(2, 6), (6, 14)], 2: [(7, 3), (3, 131)], 3: [(9, 12), (12, 20)], 5: [(13, 8), (8, 136)], 6: [(129, 132), (132, 140)], 8: [(133, 128), (128, 256)], 9: [(138, 142), (142, 150)], 11: [(143, 139), (139, 267)]}


In [6]:
#this is a dictionary of all sites in a unit cell and their corresponding chains
sites_allchains={}
sites_allchains.update(sites_2chains)
sites_allchains.update(sites_3chains)
print(sites_allchains)
print("there are", len(sites_allchains), "chains (sites) per unit cell")

{1: (1, 5), 4: (10, 15), 7: (130, 135), 10: (137, 141), 0: [(2, 6), (6, 14)], 2: [(7, 3), (3, 131)], 3: [(9, 12), (12, 20)], 5: [(13, 8), (8, 136)], 6: [(129, 132), (132, 140)], 8: [(133, 128), (128, 256)], 9: [(138, 142), (142, 150)], 11: [(143, 139), (139, 267)]}
there are 12 chains (sites) per unit cell


In [7]:
#chains and couplings within a 2x2 unit cell

J=0.5
unitcellmatrix = np.matrix([
    [0,  1,2,3,5,6,7,8,9,10,12,13,14,15,20,128,129,130,131,132,133,135,136,137,138,139,140,141,142,143,150,256,267],
    [1,  0,0,0,2,0,J,0,0,0, 0, 0, 0, 0, 0, 0,  J,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [2,  0,0,0,J,3,J,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [3,  0,0,0,0,0,3,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [5,  2,J,0,0,0,0,0,0,0, 0, J, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [6,  0,3,0,0,0,0,0,0,0, 0, 0, 3, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [7,  J,J,3,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [8,  0,0,0,0,0,0,0,0,0, 0, 3, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [9,  0,0,0,0,0,0,0,0,0, 3, J, 0, J, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [10, 0,0,0,0,0,0,0,0,0, 0, J, 0, 2, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  J,  0,  0,  0,  0,  0,  0,  0,  0],
    [12, 0,0,0,0,0,0,0,3,0, 0, 0, 0, 0, 3, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [13, 0,0,0,J,0,0,3,J,J, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [14, 0,0,0,0,3,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [15, 0,0,0,0,0,0,0,J,2, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [20, 0,0,0,0,0,0,0,0,0, 3, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [128,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0],
    [129,J,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  3,  J,  J,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [130,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  J,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [131,0,0,3,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [132,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0],
    [133,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 3,  J,  J,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [135,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  J,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  J,  0,  0,  0],
    [136,0,0,0,0,0,0,3,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  J,  0,  0,  0,  0,  0,  0],
    [137,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  J,  0,  0,  0],
    [138,0,0,0,0,0,0,0,0,J, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  J,  3,  J,  0,  0,  0],
    [139,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  3],
    [140,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  3,  0,  0,  J,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [141,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  2,  J,  0,  0,  0,  0,  0,  0,  0,  0],
    [142,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  3,  0,  0],
    [143,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  J,  0,  J,  J,  3,  0,  0,  0,  0,  0,  0,  0],
    [150,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0],
    [256,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [267,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0]
])

In [8]:
# getting a list of the qubits included in a unit cell. The first row in the big matrix is the list of qubits.
unitcellqubits=unitcellmatrix[0]
#conoverting to array
unitcellqubits=np.asarray(unitcellqubits)
#converting from floats to ints
unitcellqubits=unitcellqubits.astype(int)
unitcellqubits=unitcellqubits[0]
#deleting the 0 since it was a placeholder in the big matrix
unitcellqubits=np.delete(unitcellqubits,0)
print(unitcellqubits)
print("There are", len(unitcellqubits), "qubits per unit cell")

[  1   2   3   5   6   7   8   9  10  12  13  14  15  20 128 129 130 131
 132 133 135 136 137 138 139 140 141 142 143 150 256 267]
There are 32 qubits per unit cell


In [9]:
#checking symmetry of unit cell matrix to check for input errors
unitcellmatrix.getT().all()==unitcellmatrix.all()

True

In [10]:
#this matrix contains the couplers necessary to tile the unit cell
tilingmatrix = np.matrix([
    [0,  15,23,130,137,141,147,149,150,258,265,267,270,256,260],
    [15, 0, J, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [23, J, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [130,0, 0, 0,  0,  0,  0,  0,  0,  J,  0,  0,  0,  0,  0],
    [137,0, 0, 0,  0,  0,  0,  0,  0,  0,  J,  0,  0,  0,  0],
    [141,0, 0, 0,  0,  0,  0,  J,  0,  0,  0,  0,  0,  0,  0],
    [147,0, 0, 0,  0,  0,  0,  0,  J,  0,  0,  0,  0,  0,  0],
    [149,0, 0, 0,  0,  J,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [150,0, 0, 0,  0,  0,  J,  0,  0,  0,  0,  0,  0,  0,  0],
    [258,0, 0, J,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [265,0, 0, 0,  J,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [267,0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  J,  0,  0],
    [270,0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  J,  0,  0,  0],
    [256,0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  J],
    [260,0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  J,  0]
])

In [11]:
#checking symmetry of tiling matrix to check for input errors
tilingmatrix.getT().all()==tilingmatrix.all()

True

In [12]:
#keeping a record of the 2 qubit chains and their coupling values
couplings_2chains={}
jvalue_2chains=-2.0
for i in range(1,unitcellmatrix.shape[0]):
    for j in range(1,unitcellmatrix.shape[0]):
        if unitcellmatrix[i,j]==2:
            index1=i
            index2=j
            qubit1=unitcellmatrix[i,0]
            qubit2=unitcellmatrix[0,j]
            list=[qubit1,qubit2]
            #this gets rid of repeats. ie, (1,5) and (5,1)
            list.sort()
            couplings_2chains.update({(list[0],list[1]):jvalue_2chains})
print(couplings_2chains)
print("There are",len(couplings_2chains),"2chains per unit cell")

#keeping a record of the 3 qubit chains and their coupling values
couplings_3chains={}
jvalue_3chains=-3.0
for i in range(1,unitcellmatrix.shape[0]):
    for j in range(1,unitcellmatrix.shape[0]):
        if unitcellmatrix[i,j]==3:
            index1=i
            index2=j
            qubit1=unitcellmatrix[i,0]
            qubit2=unitcellmatrix[0,j]
            list=[qubit1,qubit2]
            #this gets rid of repeats. ie, (2,6) and (6,2)
            list.sort()
            couplings_3chains.update({(list[0],list[1]):jvalue_3chains})
print(couplings_3chains)
print("There are",len(couplings_3chains),"3chains per unit cell (when split into 2chains)")

#keeping a record of the internal couplings and their coupling values
couplings_internal={}
jvalue_internal=-1.0
for i in range(1,unitcellmatrix.shape[0]):
    for j in range(1,unitcellmatrix.shape[0]):
        if unitcellmatrix[i,j]==J:
            index1=i
            index2=j
            qubit1=unitcellmatrix[i,0]
            qubit2=unitcellmatrix[0,j]
            list=[qubit1,qubit2]
            #this gets rid of repeats. ie, (2,6) and (6,2)
            list.sort()
            couplings_internal.update({(list[0],list[1]):jvalue_internal})
print(couplings_internal)
print("There are",len(couplings_internal),"internal couplings per unit cell")

#keeping a record of the tiling couplings and their coupling values
couplings_tiling={}
jvalue_tiling=-1.0
for i in range(1,tilingmatrix.shape[0]):
    for j in range(1,tilingmatrix.shape[0]):
        if tilingmatrix[i,j]==J:
            index1=i
            index2=j
            qubit1=tilingmatrix[i,0]
            qubit2=tilingmatrix[0,j]
            list=[qubit1,qubit2]
            #this gets rid of repeats. ie, (2,6) and (6,2)
            list.sort()
            couplings_tiling.update({(list[0],list[1]):jvalue_tiling})
print(couplings_tiling)
print("There are",len(couplings_tiling),"tiling couplings")

#Dictionary with all chains, internal couplings, and tiling couplings
couplings_all_unit={}
couplings_all_unit.update(couplings_2chains)
couplings_all_unit.update(couplings_3chains)
couplings_all_unit.update(couplings_internal)
couplings_all_unit.update(couplings_tiling)

print("There are",len(couplings_all_unit),"total couplings per unit cell")

{(1.0, 5.0): -2.0, (10.0, 15.0): -2.0, (130.0, 135.0): -2.0, (137.0, 141.0): -2.0}
There are 4 2chains per unit cell
{(2.0, 6.0): -3.0, (3.0, 7.0): -3.0, (3.0, 131.0): -3.0, (6.0, 14.0): -3.0, (8.0, 13.0): -3.0, (8.0, 136.0): -3.0, (9.0, 12.0): -3.0, (12.0, 20.0): -3.0, (128.0, 133.0): -3.0, (128.0, 256.0): -3.0, (129.0, 132.0): -3.0, (132.0, 140.0): -3.0, (138.0, 142.0): -3.0, (139.0, 143.0): -3.0, (139.0, 267.0): -3.0, (142.0, 150.0): -3.0}
There are 16 3chains per unit cell (when split into 2chains)
{(1.0, 7.0): -1.0, (1.0, 129.0): -1.0, (2.0, 5.0): -1.0, (2.0, 7.0): -1.0, (5.0, 13.0): -1.0, (9.0, 13.0): -1.0, (9.0, 15.0): -1.0, (10.0, 13.0): -1.0, (10.0, 138.0): -1.0, (129.0, 133.0): -1.0, (129.0, 135.0): -1.0, (130.0, 133.0): -1.0, (135.0, 143.0): -1.0, (136.0, 140.0): -1.0, (137.0, 143.0): -1.0, (138.0, 141.0): -1.0, (138.0, 143.0): -1.0}
There are 17 internal couplings per unit cell
{(15.0, 23.0): -1.0, (130.0, 258.0): -1.0, (137.0, 265.0): -1.0, (141.0, 149.0): -1.0, (147.0, 15

In [13]:
import termcolor
from termcolor import colored

#checking that the couplers in the unit cell are present in the system

if all(coupler in sampler.edgelist for coupler in couplings_all_unit.keys())==True:
    print("all couplers in unit cell are good")
else:
    for coupler in couplings_all_unit.keys():
        if coupler in sampler.edgelist:
            print(colored(coupler, 'green'), colored('is found in list', 'green'))
        else:
            print(colored(coupler, 'red'), colored('is not found in list', 'red'))

all couplers in unit cell are good


In [14]:
#this matrix will be added on to the base unit cell matrix to tile to the right (+16)
unitcell_plus16 = np.zeros(unitcellmatrix.shape)
for i in range(1,unitcellmatrix.shape[0]):
    unitcell_plus16[0][i]=16
    unitcell_plus16[i][0]=16
unitcell_plus16=np.asmatrix(unitcell_plus16)

#this matrix will be added on to the base unit cell matrix to tile down (+256)
unitcell_plus256 = np.zeros(unitcellmatrix.shape)
for i in range(1,unitcellmatrix.shape[0]):
    unitcell_plus256[0][i]=256
    unitcell_plus256[i][0]=256
unitcell_plus256=np.asmatrix(unitcell_plus256)

#print(unitcell_plus16,unitcell_plus16.shape)
#print(unitcell_plus256,unitcell_plus256.shape)

In [15]:
#this matrix will be added on to the base tiling matrix to tile to the right (+16)
tiling_plus16 = np.zeros((15,15))
for i in range(1,tilingmatrix.shape[0]):
    tiling_plus16[0][i]=16
    tiling_plus16[i][0]=16
tiling_plus16=np.asmatrix(tiling_plus16)

#this matrix will be added on to the base tiling matrix to tile down (+256)
tiling_plus256 = np.zeros((15,15))
for i in range(1,tilingmatrix.shape[0]):
    tiling_plus256[0][i]=256
    tiling_plus256[i][0]=256
tiling_plus256=np.asmatrix(tiling_plus256)

#print(tiling_plus16,tiling_plus16.shape)
#print(tiling_plus256,tiling_plus256.shape)

In [16]:
#checking the shape of the unitcell/tiling matrix vs the translation matrices

print((unitcellmatrix.shape,tilingmatrix.shape),(unitcell_plus16.shape,tiling_plus16.shape))
print((unitcellmatrix.shape,tilingmatrix.shape),(unitcell_plus256.shape,tiling_plus256.shape))

((33, 33), (15, 15)) ((33, 33), (15, 15))
((33, 33), (15, 15)) ((33, 33), (15, 15))


In [17]:
#function that tiles a unitcell to the right once
def tile_right(unitcellmat,tilingmat):
    
    tileright_unitcell=unitcellmat+unitcell_plus16
    tileright_tiling=tilingmat+tiling_plus16
    
    return tileright_unitcell,tileright_tiling


#function that tiles a unitcell down once
def tile_down(unitcellmat,tilingmat):
    
    tiledown_unitcell=unitcellmat+unitcell_plus256
    tiledown_tiling=tilingmat+tiling_plus256
    
    return tiledown_unitcell,tiledown_tiling

In [18]:
#NxN tiling: This function tiles in +x direction N-1 times and -y direction N-1 times. 
#Output is basically a NxN lattice
def NxN_tile(N):
    #making a registry of unitcells. The first entry is the base unitcell
    unitcellregistry=[]
    unitcellregistry.append(unitcellmatrix)
    
    #making a registry of tilings. The first entry is the base tiling
    tilingregistry=[]
    tilingregistry.append(tilingmatrix)
    
    
    #tiling to the right N-1 times, and appending the registries with the new tiled matrices
    for i in range(N-1):
        newmatsright=tile_right(unitcellregistry[i],tilingregistry[i])
        unitcellregistry.append(newmatsright[0])
        tilingregistry.append(newmatsright[1])
        
    #Now there is a row of N unit cells in the registries. For each unit cell in this row, we tile down N-1 times.
    #a list containing the row to be tiled down. Initialize with 1st row. It will be updated with each iteration
    tobetiled_down=[]
    for i in range(N):
        tobetiled_down.append((unitcellregistry[i],tilingregistry[i]))
    
    
    
    for i in range(N-1):
        
        newrow=[]
        for j in tobetiled_down:
            #tile down each unit cell in the preceding row
            newmatsdown=tile_down(j[0],j[1])
            
            #update the registries with each new matrix
            unitcellregistry.append(newmatsdown[0])
            tilingregistry.append(newmatsdown[1])
            
            #populate newrow with the new matrices
            newrow.append(newmatsdown)
            #update the to be tiled down list to be used in the new iteration
            tobetiled_down=newrow
            
    
    
    return unitcellregistry, tilingregistry

In [19]:
#Num is the desired number of unit cells in one dimension (Defined this way so that we can use this number later)
Num=8
x=NxN_tile(Num)
#x

In [20]:
#setting j values here
jvalue_2chains=-2.0   
jvalue_3chains=-3.0
jvalue_internal=-1.0
jvalue_tiling=-1.0


#input both registries and output a list of dictionaries of the form (qubiti,qubitj: jvalue)
def getcouplings(ucreg,tilreg):
    
    couplings_2chains={}    
    couplings_3chains={}   
    couplings_internal={}
    couplings_tiling={}
    
    
    
    for matrix in ucreg:
    
     
        #keeping a record of the 2 qubit chains and their coupling values
        for i in range(1,unitcellmatrix.shape[0]):
            for j in range(1,unitcellmatrix.shape[0]):
                if matrix[i,j]==2:
                    index1=i
                    index2=j
                    qubit1=matrix[i,0]
                    qubit2=matrix[0,j]
                    list=[qubit1,qubit2]
                    #this gets rid of repeats. ie, (1,5) and (5,1)
                    list.sort()
                    couplings_2chains.update({(list[0],list[1]):jvalue_2chains})
    
        #keeping a record of the 3 qubit chains and their coupling values
        for i in range(1,unitcellmatrix.shape[0]):
            for j in range(1,unitcellmatrix.shape[0]):
                if matrix[i,j]==3:
                    index1=i
                    index2=j
                    qubit1=matrix[i,0]
                    qubit2=matrix[0,j]
                    list=[qubit1,qubit2]
                    #this gets rid of repeats. ie, (2,6) and (6,2)
                    list.sort()
                    couplings_3chains.update({(list[0],list[1]):jvalue_3chains})

        #keeping a record of the internal couplings and their coupling values
        for i in range(1,unitcellmatrix.shape[0]):
            for j in range(1,unitcellmatrix.shape[0]):
                if matrix[i,j]==J:
                    index1=i
                    index2=j
                    qubit1=matrix[i,0]
                    qubit2=matrix[0,j]
                    list=[qubit1,qubit2]
                    #this gets rid of repeats. ie, (2,6) and (6,2)
                    list.sort()
                    couplings_internal.update({(list[0],list[1]):jvalue_internal})
        
        
        
    for matrix in tilreg:

    
        #keeping a record of the tiling couplings and their coupling values
        for i in range(1,tilingmatrix.shape[0]):
            for j in range(1,tilingmatrix.shape[0]):
                if matrix[i,j]==J:
                    index1=i
                    index2=j
                    qubit1=matrix[i,0]
                    qubit2=matrix[0,j]
                    list=[qubit1,qubit2]
                    #this gets rid of repeats. ie, (2,6) and (6,2)
                    list.sort()
                    couplings_tiling.update({(list[0],list[1]):jvalue_tiling})
                    

    #Dictionary with all chains, internal couplings, and tiling couplings
    all_couple_values={}
    all_couple_values.update(couplings_2chains)
    all_couple_values.update(couplings_3chains)
    all_couple_values.update(couplings_internal)
    all_couple_values.update(couplings_tiling)
    
    #print(couplings_2chains)
    print("There are",len(couplings_2chains),"2chains")
    
    #print(couplings_3chains)
    print("There are",len(couplings_3chains),"3chains (when split into 2chains)")
            
    #print(couplings_internal)
    print("There are",len(couplings_internal),"internal couplings")
    
    #print(couplings_tiling)
    print("There are",len(couplings_tiling),"tiling couplings")
    
    return all_couple_values

In [21]:
couplers=getcouplings(x[0],x[1])
#couplers

There are 256 2chains
There are 1024 3chains (when split into 2chains)
There are 1088 internal couplings
There are 448 tiling couplings


In [22]:
#checking number of couplers
len(couplers)

2816

In [23]:
#getting the tuples from the coupler dictionary
tuples_only=[]
for edge in couplers.keys():
    tuples_only.append(edge)
    
#print(tuples_only)

In [24]:
#checking if all the couplers being used are present in the qpu. 
#If one is missing for some reason, it will be removed

while all(edge in sampler.edgelist for edge in tuples_only)==False:
    for edge in tuples_only:
        if edge not in sampler.edgelist:
            print(colored(edge, 'red'), colored('is not found in list, removing', 'red'))

            tuples_only.remove(edge)
            couplers.pop(edge)

    if all(edge in sampler.edgelist for edge in tuples_only)==True:
            print("done, all couplers are good")

(1730.0, 1735.0) is not found in list, removing
(38.0, 46.0) is not found in list, removing
(124.0, 132.0) is not found in list, removing
(254.0, 262.0) is not found in list, removing
(380.0, 388.0) is not found in list, removing
(510.0, 518.0) is not found in list, removing
(521.0, 524.0) is not found in list, removing
(540.0, 548.0) is not found in list, removing
(636.0, 644.0) is not found in list, removing
(766.0, 774.0) is not found in list, removing
(892.0, 900.0) is not found in list, removing
(1022.0, 1030.0) is not found in list, removing
(1148.0, 1156.0) is not found in list, removing
(1278.0, 1286.0) is not found in list, removing
(1404.0, 1412.0) is not found in list, removing
(1534.0, 1542.0) is not found in list, removing
(1723.0, 1727.0) is not found in list, removing
(1660.0, 1668.0) is not found in list, removing
(1790.0, 1798.0) is not found in list, removing
(1801.0, 1804.0) is not found in list, removing
(1920.0, 2048.0) is not found in list, removing
(1931.0, 2059.

In [25]:
#rechecking all couplers before annealing
if all(edge in sampler.edgelist for edge in tuples_only)==True:
    print("all couplers are good")

all couplers are good


In [32]:
#this cell is a simulated anneal. ouputs a "response"
#make sure either this cell or the cell below is commented out

# h={}
# J=couplers
# model = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.SPIN)

# sampler2 = dimod.SimulatedAnnealingSampler()
# response = sampler2.sample(model, num_reads=10)
# print(response)

In [33]:
#this cell is a quantum anneal. ouputs a "response" and opens up the inspector
#make sure either this cell or the cell above is commented out

# h={}
# J=couplers
# response = sampler.sample_ising(h, J, num_reads=800)

# print(response)
# dwave.inspector.show(response)

In [28]:
#a function for setting a document name for storing the response

from datetime import datetime

def DocName(jvalue_internal,Num):
    str1=str(Num)
    
    if jvalue_internal<0:
        str2="NegJ"
    else:
        str2="PosJ"
    
    now = datetime.now()
    str3 = now.strftime("%m-%d-%y__%H-%M-%S")
    
    Documentname=str1+"x"+str1+"Kagome"+str2+"__"+str3
    
    return Documentname

In [29]:
#converting response to pandas dataframe, and then storing as a csv

# Name=DocName(jvalue_internal,Num)

# from pathlib import Path  
# filepath = Path("/Users/prathunarasimhan/Documents/QIQC/KagomeNxN_runs/",Name) 

# to_pandas=response.to_pandas_dataframe(sample_column=True)
# #print(to_pandas)

# to_pandas.to_csv(filepath)